# Interroger votre base de connaissances

In [ ]:
%pip install -qU wget

In [18]:
import json
import os
import re
import requests

import wget

In [ ]:
base_url = "https://albert.api.etalab.gouv.fr/v1"
api_key = "YOUR_API_KEY"

session = requests.session()
session.headers = {"Authorization": f"Bearer {api_key}"}

Commencez par télécharger le document qui va vous servir de base de connaissance. Nous prenons ici comme exemple le dataset des expériences d'usagers provenant du site de la Transformation Publique.

In [19]:
doc_url = "https://opendata.plus.transformation.gouv.fr/api/explore/v2.1/catalog/datasets/export-expa-c-riences/exports/json"
file_path = "my_database.json"

if not os.path.exists(file_path):
    wget.download(doc_url, out=file_path)

Le format du fichier doit être JSON, ici nous n'avons pas besoin de le convertir. En revanche, il est possible que le fichier ne respecte pas la structure définit dans la documentation de l'API.

De plus le fichier ne doit pas dépasser 20MB, il est donc nécessaire de le découper en plusieurs fichiers.

In [20]:
file = json.load(open(file_path))

print("Exemple de fichier :", file[0])
print("Poids du fichier :", os.path.getsize(file_path) // 1024, "MB")

Exemple de fichier : {'id_experience': 280027, 'etat_experience': 'A traiter', 'statut_de_l_experience': 'Expérience en attente de réponse', 'ecrit_le': '2019-06-20', 'date_de_publication': '2019-06-21', 'pseudonyme_usager': 'FLORIAN H.', 'titre': 'ANAH - programme habiter mieux', 'description': 'Nous avons déposé un dossier complet à l\'Anah en date du 18 décembre 2018 mais depuis cette date, il n\' y a pas encore eu de "commission" pour valider notre dossier. Fin mars, on nous avait annoncé une commission pour le 16 juin et hier, nous avons appris qu\'elle a été, une énième fois, reportée... sans avoir plus d\'infos.Pourquoi une telle situation ?', 'poste_comme': 'La personne concernée', 'ressenti_usager': 'Négatif', 'code_insee_departement_usager': None, 'intitule_departement_usager': None, 'intitule_region_usager': None, 'code_insee_region_usager': None, 'pays': 'France', 'id_typologie_1': 'vAjIgQajQN', 'intitule_typologie_1': 'ANAH', 'code_postal_typologie_1': None, 'pays_typologi

Commençons par changer la structure du fichier. Celle-ci doit être une liste de dictionnaire avec comme clés `title`, `text` et `metadata` (cette dernière est optionnelle).

Nous profitons également de cette étape pour nettoyer le texte d'artefacts et le rendre plus proche de ce que nous voulons obtenir.

Pour l'exemple, nous allons importer que les 120 premiers documents.


In [21]:
formated_file = list()
sample_size = 120
for document in file[:sample_size]:
    titre = document.get("titre", "")
    description = document.get("description", "")
    intitule = document.get("intitule_typologie_1", "")
    reponse = document.get("reponse_structure_1", "")

    text = re.sub(r"([.,;:!?])([^\s\d])", r"\1 \2", description)  # Add space after punctuation
    text = re.sub(r"[\xa0\u00a0\r]", " ", text)  # Remove special characters
    text = re.sub(r"&nbsp;", " ", text)
    text = re.sub(r"\,(?!\s)", ". ", text, count=1)  # Add a space after the first "," if not already followed by a space.

    formated_file.append(
        {
            "title": titre,
            "text": f"{titre} {text} {intitule}",
            "metadata": {"question": text, "answer": reponse, "titre": titre, "intitule": intitule},
        }
    )

print(f"Exemple de fichier : {formated_file[0]}")

Exemple de fichier : {'title': 'ANAH - programme habiter mieux', 'text': 'ANAH - programme habiter mieux Nous avons déposé un dossier complet à l\'Anah en date du 18 décembre 2018 mais depuis cette date, il n\' y a pas encore eu de "commission" pour valider notre dossier. Fin mars, on nous avait annoncé une commission pour le 16 juin et hier, nous avons appris qu\'elle a été, une énième fois, reportée. .. sans avoir plus d\'infos. Pourquoi une telle situation ? ANAH', 'metadata': {'question': 'Nous avons déposé un dossier complet à l\'Anah en date du 18 décembre 2018 mais depuis cette date, il n\' y a pas encore eu de "commission" pour valider notre dossier. Fin mars, on nous avait annoncé une commission pour le 16 juin et hier, nous avons appris qu\'elle a été, une énième fois, reportée. .. sans avoir plus d\'infos. Pourquoi une telle situation ?', 'answer': "Bonjour Florian H., et merci de votre témoignage.A ce jour, l'Agence nationale de l'habitat n'a pas encore la possibilité de vo


Nous pouvons maintenant importer le fichier dans la base vectorielle d'Albert à l'aide de l'API.

Pour commencer, nous créons une collection nommée `tutorial`. Pour cela nous effectuons une requête GET sur l'endpoint `/v1/models` afin d'obtenir la liste des modèles disponibles et définissons le modèle d'embeddings à utiliser.

In [24]:
response = session.get(f"{base_url}/models")
response = response.json()
model = [model for model in response["data"] if model["type"] == "text-embeddings-inference"][0]["id"]

print(f"Embeddings model: {model}")

Embeddings model: BAAI/bge-m3


In [25]:
collection = "tutorial"

response = session.post(f"{base_url}/collections", json={"name": collection, "model": model})
response = response.json()
collection_id = response["id"]
print(f"Collection ID: {collection_id}")

Collection ID: 3320eea7-2074-409d-a3e4-797444e161ee


Nous pouvons maintenant importer le fichier dans la base vectorielle d'Albert à l'aide de l'API.


In [26]:
files = {"file": (os.path.basename(file_path), open(file_path, "rb"), "application/pdf")}
data = {"request": '{"collection": "%s"}' % collection_id}
response = session.post(f"{base_url}/files", data=data, files=files)
print(response.json())

{'detail': 'File size limit exceeded'}


La taille du fichier dépasse 20MB, il est donc nécessaire de le découper en plusieurs fichiers.

In [27]:
batch = 64
for i in range(0, len(formated_file), batch):
    batch_file = formated_file[i : i + batch]

    batch_file_path = f"tmp_{i}.json"
    json.dump(batch_file, open(batch_file_path, "w"))
    assert os.path.getsize(batch_file_path) < 20 * 1024 * 1024

    files = {"file": (os.path.basename(batch_file_path), open(batch_file_path, "rb"), "application/json")}
    data = {"request": '{"collection": "%s"}' % collection_id}
    response = session.post(f"{base_url}/files", data=data, files=files)
    assert response.status_code == 201, "Erreur lors de l'importation du fichier"
    os.remove(batch_file_path)

Nous pouvons maintenant vérifier que les fichiers ont bien été importés à l'aide du endpoint GET `/v1/documents`.

In [32]:
response = session.get(f"{base_url}/documents/{collection_id}")
response.json()
files = response.json()["data"]
print(len(files))

120


Enfin nous pouvons supprimer les fichiers importés.


In [33]:
response = session.delete(f"{base_url}/collections/{collection_id}")